# Setup

In [1]:
using ProgressMeter, Distributed
# Add workers for parallel processing
prs = addprocs(;topology=:master_worker, exeflags="-O3")

@everywhere using Pkg; @everywhere Pkg.activate(".")

# Load relevant code on all processes
using ArmRoleStability
@everywhere using ArmRoleStability

┌ Info: Recompiling stale cache file /home/allen/.julia/compiled/v1.0/ArmRole/O2XaH.ji for ArmRole [f6a2469e-950c-11e8-131e-8998aaf96874]
└ @ Base loading.jl:1190


In [2]:
# Define data location for all processes
if Sys.islinux()
    diskLoc = "/media/allen/Seagate Backup Plus Drive/"
#     diskLoc = "/media/allen/My Book - Master/"
elseif Sys.iswindows()
   diskLoc = "F:\\"
end

rootdir = joinpath(diskLoc, "projects", "Arm-role-stability")

# Read all perturbations
sstrials = readsstrials(rootdir);

ReentrantLock(nothing, Condition(Any[]), 0)

In [3]:
[ (Subject=r[1], numtrials=length(r[2])) for r in [ (sub, findall(t -> t.subject == sub, sstrials)) for sub in 1:15 ] ]

15-element Array{NamedTuple{(:Subject, :numtrials),Tuple{Int64,Int64}},1}:
 (Subject = 1, numtrials = 6) 
 (Subject = 2, numtrials = 6) 
 (Subject = 3, numtrials = 6) 
 (Subject = 4, numtrials = 6) 
 (Subject = 5, numtrials = 6) 
 (Subject = 6, numtrials = 6) 
 (Subject = 7, numtrials = 6) 
 (Subject = 8, numtrials = 6) 
 (Subject = 9, numtrials = 6) 
 (Subject = 10, numtrials = 6)
 (Subject = 11, numtrials = 6)
 (Subject = 12, numtrials = 6)
 (Subject = 13, numtrials = 6)
 (Subject = 14, numtrials = 6)
 (Subject = 15, numtrials = 6)

## Main analysis

In [ ]:
# Setup progressmeter and lock
pdesc = "Processing data... "
p = Progress(length(sstrials)+1; desc=pdesc, barglyphs=BarGlyphs("[=>.]"))
uplock = ReentrantLock()

In [4]:
# Update the progressmeter in a thread-safe manner
@everywhere function updateprogress()
    lock(uplock)
    next!(p)
    unlock(uplock)
    nothing
end

# This is used by the workers
@everywhere function analyzeandupdate(trial)
    numstrides = 125
    atrial = analyzetrial(trial, numstrides)
    
    # Tell the master process to update the progressmeter
    remotecall_wait(updateprogress,1)
    return atrial
end

In [5]:
# Fit all the perturbations, don't quit on an error, add it to the results
next!(p)
analyzedtrials = pmap(analyzeandupdate, sstrials; on_error=identity)
finish!(p)

Processing data... 100%[================================] Time: 0:01:18


In [6]:
using Biomechanics

In [8]:
# Check to see if any perturbations failed (to go back and see why they failed)
badtrials = findall(x -> !isa(x, AnalyzedSegment), analyzedtrials)
if !isempty(badtrials)
    @show badtrials
    @show analyzedtrials[badtrials]
    variables = [
        :lambdaS,
        :left_steplavg,
        :left_steplstd,
        :right_steplavg,
        :right_steplstd,
        :stepwavg,
        :stepwstd,
        :rsho_avgrom,
        :lsho_avgrom,
        :rsho_stdrom,
        :lsho_stdrom,
        :swingasym,
        :stepasym_spatial,
        :stepasym_temporal,
        :msdcrp_lsho_rhip,
        :msdcrp_rsho_lhip
    ]
    # NAN the bad trials
    analyzedtrials[badtrials] = [ AnalyzedSegment(Segment(trial, Dict{Symbol,Vector}(), ArmRoleSeg()), 
                                                          Dict{Symbol,Any}(( (var, NaN) for var in variables ))) 
                                  for trial in sstrials[badtrials] ]
end

In [9]:
# We don't need the other workers anymore
rmprocs(prs)

Task (done) @0x00007f4d616679d0

## Results printing setup

In [10]:
using Statistics, Dates, DelimitedFiles

In [11]:
# Setup loop variables
variables = [
    :lambdaS,
    :left_steplavg,
    :left_steplstd,
    :right_steplavg,
    :right_steplstd,
    :stepwavg,
    :stepwstd,
    :rsho_avgrom,
    :lsho_avgrom,
    :rsho_stdrom,
    :lsho_stdrom,
    :swingasym,
    :stepasym_spatial,
    :stepasym_temporal,
    :msdcrp_lsho_rhip,
    :msdcrp_rsho_lhip
]
armconds = [ :none, :norm, :excess ]
shortarms = Dict(
    :norm => "_nr",
    :none => "_no",
    :excess => "_ex"
)
symconds = [ :sym, :asym ]
shortsym = Dict(
    :sym => "_sy,",
    :asym => "_as,"
)

subs = 1:15
numsubs = length(subs)
header = 4

# Initialize the results string
results = Vector{String}(undef, 1)
results[1] = "Analysis of trunk motion during steady-state walking\nGenerated: $(now())\n"

"Analysis of trunk motion during steady-state walking\nGenerated: 2019-02-04T14:20:56.387\n"

In [12]:
# Create views for repeatedly used parts of the results

# subresults = view(results, (header+1):length(results))

# Loop through all variables and conditions
# Beginning with the variables will keep all the variables chunked together, this will make
# the analysis in SPSS easier (all the relevant columns will be somewhat adjacent, limiting
# the need for lateral scrolling )
io = IOBuffer()
for vari in eachindex(variables)
    subresults = fill(",", 4)
    R = collect(1:15)
    _r = Array{Float64}(undef, numsubs)
    for arms in eachindex(armconds), symmetry in eachindex(symconds)
        # Only print the variable/condition if it is the first of its type; this will allow a
        # merged cell to be created to encompass all the columns below with that condition
        subresults[1] *= prod([ arms, symmetry ] .== ones(Int,2)) ? string(variables[vari])*"," : ","
        subresults[2] *= (symmetry == one(Int)) ? string(armconds[arms], ",") : ","
        subresults[3] *= string(symconds[symmetry], ",")
        subresults[4] *= string(variables[vari])*shortarms[armconds[arms]]*shortsym[symconds[symmetry]]

        # Grab all the perturbations for this combination of conditions
        relevant = findall(analyzedtrials) do ass
            ass.s.trial.conds[:arms] == armconds[arms] &&
            ass.s.trial.conds[:sym] == symconds[symmetry]
        end

        # Add the average value for the current variable (eg A, B, etc.) to each subject
        # Nasty/ugly because it is so dense/concise; I think it is better than how verbose the
        # alternative would be
        # The array comprehension runs the inner nastiness for each subject
        
        for sub in subs
            # The generator expression is outputting the current variable (eg A, B, etc) for all
            # the relevant perturbations for the current subject
            # The function chain averages the array from the generator expression, and then
            # creates a string representation
            t = filter(ass -> ass.s.trial.subject == sub, analyzedtrials[relevant])[1].results[variables[vari]]

            _r[sub] = t
        end
        
        R = [ R _r ]
    end

    writedlm(io, R, ',')

    results = [results; "\n\n"; subresults; String(take!(io))]
end

## Write to file

In [13]:
(path, io) = mktemp()

for line in results
    println(io, line)
end

close(io)

resfn = joinpath(rootdir, "results", "steady-state_test.csv")

mv(path, resfn; force=true)

"/media/allen/Seagate Backup Plus Drive/projects/Arm-role-stability/results/cezar-steady-state-step_spatiotemporal.csv"